In [ ]:
# CS-340 Project Two Dashboard
# Course: CS-340-11283
# Author: Nik Myers
# Instructor: Robert Kayl
# Date: 2025-04-18

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import Dash, dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD module for MongoDB
from aac_crud import CRUD

###########################
# Data Manipulation / Model
###########################
# MongoDB connection variables
USERNAME = "aacuser"
PASSWORD = "access-control"  # Don't use plaintext in production code!
HOST = "localhost"  # MongoDB server hostname [nv-desktop-services.apporto.com]
PORT = 27017  # MongoDB server port 27017 (local), 34212 (apporto)
DB = "AAC"  # Database name
COL = "animals"  # Collection name
AUTHDB = "admin"

# Connect to database via CRUD Module
db = CRUD(USERNAME, PASSWORD, HOST, PORT, AUTHDB)

# Read the data from the database and create a DataFrame
df = pd.DataFrame.from_records(db.find_documents(DB, COL, {}))

# Drop the '_id' column if it exists
df.drop(columns=['_id'], inplace=True)

# Options for filtering
RESCUE_OPTIONS = [
    {"label": "All", "value": "All"},
    {"label": "Water Rescue", "value": "Water Rescue"},
    {"label": "Mountain/Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
    {"label": "Disaster/Individual Tracking", "value": "Disaster or Individual Tracking"},
]

# Breed lists for each rescue category
WATER_BREEDS = ["Labrador Retriever", "Newfoundland", "Golden Retriever"]
MOUNTAIN_BREEDS = ["German Shepherd", "Siberian Husky", "Border Collie"]
DISASTER_BREEDS = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]

#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

# Grazioso Salvare logo as base64 encoded image
image_filename = 'assets/gs_logo.png'  # GS logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.H1('CS-340 Dashboard', style={'textAlign': 'center'}),
    html.Hr(),
    html.Div([
        html.Img(id='logo-image',
                 src='data:image/png;base64,{}'.format(encoded_image.decode()),
                 alt='Grazioso Salvare Logo', height='64px'),
        html.Span("Built by Nik Myers", style={"marginLeft": "auto", "fontStyle": "italic"}),
    ], style={"display": "flex", "alignItems": "center", "marginBottom": "1rem"}),
    html.Div(
        dcc.Dropdown(
            id="rescue-filter",
            options=RESCUE_OPTIONS,
            value="All",
            clearable=False,
            style={"width": "50%"}
        ),
    ),
    html.Hr(),
    html.Div([
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict('records'),
            # Sets up features for user interaction
            editable=False,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            column_selectable=False,
            row_selectable="single",
            row_deletable=False,
            selected_columns=[],
            selected_rows=[0],
            page_action="native",
            page_current=0,
            page_size=10,
            style_table={'overflowX': 'auto'},
        ), ], id='table-container',
        style={'display': 'block', 'maxWidth': '100%', 'overflow': 'scroll'}),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display': 'flex', 'minHeight': '500px', 'justifyContent': 'space-between'}, children=[
        html.Div(
            id='graph-id',
            className='chart-container',
        ),
        html.Div(
            id='map-id',
            className='map-container',
        )
    ])
])


#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id', 'data'),
              [Input('rescue-filter', 'value')])
def update_dashboard(filter_type):
    """
    Filter the data table based on the selected rescue type using MongoDB queries.
    """
    if filter_type == 'All':
        documents = db.find_documents(DB, COL, {})
    elif filter_type == 'Water Rescue':
        query = {"animal_type": "Dog", "breed": {"$in": WATER_BREEDS}}
        documents = db.find_documents(DB, COL, query)
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {"animal_type": "Dog", "breed": {"$in": MOUNTAIN_BREEDS}}
        documents = db.find_documents(DB, COL, query)
    elif filter_type == 'Disaster or Individual Tracking':
        query = {"animal_type": "Dog", "breed": {"$in": DISASTER_BREEDS}}
        documents = db.find_documents(DB, COL, query)
    else:
        documents = db.find_documents(DB, COL, {})

    # Convert the documents to a DataFrame and drop the '_id' column if it exists
    try:
        dff = pd.DataFrame.from_records(documents)
        if '_id' in dff.columns:
            dff.drop(columns=['_id'], inplace=True)
        return dff.to_dict('records')
    except Exception as e:
        print("[update_dashboard]: Error converting documents to DataFrame:\t", e, end="\n\n")
        return []


# Display the breeds of animal based on the quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    """
    Update the breed distribution pie chart based on the current table view.
    """
    if viewData is None:
        return None
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return html.Div("No data to display.")

    fig = px.pie(dff, names='breed', title='Breed Distribution')

    return [dcc.Graph(figure=fig)]


# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Map callback
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if index is None:
        row = 0
    else:
        row = index[0]

    # Returns a map of Austin, TX with a marker for the selected row
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id='base-layer-id'),
                dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                          children=[
                              dl.Tooltip(dff.iloc[row,4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[row,9])
                              ])
                          ])
            ])
    ]


# Run the app
app.run()

